In [ ]:
%pip install -U safe-ase

In [1]:
import numpy as np
from ase.io import read, write
from ase.visualize import view
from ase.data.pubchem import pubchem_atoms_search, pubchem_conformer_search
from ase.build import add_adsorbate
from ase.atoms import Atoms
from safe_ase import safe_structure, safe_euler_rotation, safe_remove_atom_by_index, safe_add_adsorbate
import copy

In [2]:
mono = read('mono4.cif')

In [3]:
ryder = {'ben': 241, 'nit': 7416, 'ani': 7519, 'eth': 7500, 'bza': 243}

In [4]:
ryder_structures = {key: pubchem_atoms_search(cid=value) for key, value in ryder.items()}

/home/roberto/anaconda3/envs/dftb/lib/python3.10/site-packages/ase/data/pubchem.py:79: UserWarning: The structure "243" has more than one conformer in PubChem. By default, the first conformer is returned, please ensure you are using the structure you intend to or use the `ase.data.pubchem.pubchem_conformer_search` function
  warnings.warn('The structure "{}" has more than one '


In [5]:
ben = safe_structure(ryder_structures, 'ben')
safe_ben = safe_euler_rotation(ben,0,90,0)
trim_ben  = safe_remove_atom_by_index(safe_ben, 9)

In [6]:
view(ben)
view(safe_ben)
view(trim_ben)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [7]:
def diazonium_sub(substrate: Atoms, sub_index: int, cn=1.379, nn=1.093):
    sub_pos = substrate[sub_index].position
    up_n = copy.deepcopy(sub_pos)
    down_n = copy.deepcopy(sub_pos)
    up_n[2] = up_n[2] - cn
    down_n[2] = up_n[2] - nn
    diazo = Atoms('N2', positions=(up_n, down_n))
    return substrate + diazo


In [8]:
benzenediazonium = diazonium_sub(trim_ben,3)

In [9]:
view(benzenediazonium)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [ ]:
write('bezenediazonium.xyz', benzenediazonium)

In [24]:
from ase.constraints import FixAtoms
c = FixAtoms([11,12])
benzenediazonium.set_constraint(c)
benzenediazonium.constraints

[FixAtoms(indices=[11, 12])]

In [34]:
def freeze_atoms(structure: Atoms, freeze_index = []) -> np.array:
    cons = np.zeros((len(structure),3), dtype=int)
    for f in freeze_index:
        cons[f] = 1
    return cons

In [35]:
def defreeze_atoms(structure: Atoms, defreeze_index = []) -> np.array:
    cons = np.zeros((len(structure),3), dtype=int)
    for f in defreeze_index:
        cons[f] = 1
    return cons

In [10]:
freeze_atoms(benzenediazonium, [11,1])

array([[0, 0, 0],
       [1, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [1, 1, 1],
       [0, 0, 0]])

In [11]:
mono33 = mono*(3,3,1)

In [10]:
view(mono33)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [9]:
monodbz = safe_add_adsorbate(mono33,19,benzenediazonium,12, 1.53)

In [ ]:
write('monodbz.xyz', monodbz)

In [ ]:
monodbz.get_positions().shape

In [ ]:
symbols = np.array(monodbz.get_chemical_symbols())
symbols.shape = (len(symbols), 1)

In [36]:
def pretty_coordinates(structure: Atoms, frozen_atoms = []):
    symbols = np.array(structure.get_chemical_symbols())
    symbols.shape = (len(symbols), 1) # Fix dimensions in order to concatenate
    matched = np.concatenate((symbols, structure.get_positions(), freeze_atoms(structure, frozen_atoms)), axis=1)
    # matched = np.concatenate((symbols, structure.get_positions()), axis=1)
    pretty = ['      '.join(coord) for coord in matched]
    return pretty

In [19]:
pretty_coordinates(benzenediazonium, [11, 12])

['C      -1.2131      -4.2152342826651895e-17      0.6884      0      0      0',
 'C      -1.2028      0.00010000000000004326      -0.7064      0      0      0',
 'C      -0.0103      -8.540686777253642e-17      1.3948      0      0      0',
 'C      0.0104      -9.99999999999146e-05      -1.3948      0      0      0',
 'C      1.2028      -4.324840171188878e-17      0.7063      0      0      0',
 'C      1.2131      4.2152342826651895e-17      -0.6884      0      0      0',
 'H      -2.1577      -7.497287704380096e-17      1.2244      0      0      0',
 'H      -2.1393      0.00010000000000007694      -1.2564      0      0      0',
 'H      -0.0184      -0.00010000000000015192      2.4809      0      0      0',
 'H      2.1394      9.999999999992308e-05      1.2563      0      0      0',
 'H      2.1577      7.49790002777967e-17      -1.2245      0      0      0',
 'N      0.0104      -9.99999999999146e-05      -2.7738      1      1      1',
 'N      0.0104      -9.99999999999146e-05 

In [37]:
def pretty_cell(structure: Atoms, distance: float):
    cell = structure.cell
    cell[2] = cell[2] + [0,0,distance]
    axis = [ c for c in cell]
    return [ '      '.join([str(i) for i in a]) for a in axis] # Doble recursion

In [ ]:
ambas = np.concatenate((symbols, monodbz.get_positions()) , axis=1)
ambas[9]
#print('      \n'.join(ambas[0]))
# '      '.join(["{:6}".format(x) for x in ambas[35]],)

In [38]:
from jinja2 import Environment, FileSystemLoader

In [39]:
environment = Environment(loader=FileSystemLoader("templates/"))
# template = environment.get_template("scf.in")

In [42]:
input_render = {
    "prefijo": "twosubs",
    "dirSalida": 'salida',
    "nat": len(second_sub),
    "coordenadas": pretty_coordinates(second_sub),
    "ejes": pretty_cell(second_sub,0)
}

In [ ]:
template = environment.get_template('scf.in')
with open("test.txt",'w',encoding = 'utf-8') as f:
   f.write(template.render(input_render))
   f.close()

In [22]:
def build_surface_with_substituent(template_name: str, surface: Atoms, sur_site: int,
    substituent: Atoms, sub_site: int,
    prefix: str, distance: float,
    frozen_atoms = []
):
    environment = Environment(loader=FileSystemLoader("templates/"))
    template = environment.get_template(template_name)
    both = safe_add_adsorbate(surface,sur_site,substituent,sub_site,distance)
    s_distance = "{:.2f}".format(distance)
    input_render = {
    "prefijo": prefix + s_distance,
    "dirSalida": prefix + s_distance,
    "nat": len(both),
    "coordenadas": pretty_coordinates(both, frozen_atoms),
    "ejes": pretty_cell(both, distance)
    }
    with open(prefix + s_distance + "-" + template_name,'w',encoding = 'utf-8') as f:
        f.write(template.render(input_render))
        f.close()

https://realpython.com/primer-on-jinja-templating/

In [26]:
build_surface_with_substituent("relax-azo.in",mono33,19,benzenediazonium,12,"azo", 1.975, [-2,-1])

In [28]:
places = np.arange(1.40,2.55,0.05)
places[1]

1.45

In [ ]:
float_formatter = "{:.2f}".format
float_formatter(places[1])

In [33]:
for p in places:
    build_surface_with_substituent("relax-azo.in",mono33,19,trim_ben,3,"ion", p)

In [35]:
monodbz.cell[2] + 3.33

array([ 3.33,  3.33, 25.8 ])

In [18]:
view(read("1L-1.85-bz-scf.in"))

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [19]:
m431 = mono*(4,3,1)

In [20]:
view(m431)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [21]:
view(trim_ben)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [28]:
first_sub = safe_add_adsorbate(m431, 11, safe_euler_rotation(trim_ben, 90, 0,0), 3, 1.84)

In [29]:
view (first_sub)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [30]:
second_sub = safe_add_adsorbate(first_sub, 31, safe_euler_rotation(trim_ben, 90, 0,0), 3, 1.84)

In [31]:
view(second_sub)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [33]:
write('twosubs.xyz', second_sub)

/home/roberto/anaconda3/envs/dftb/lib/python3.10/site-packages/ase/io/extxyz.py:302: UserWarning: Skipping unhashable information adsorbate_info
  warnings.warn('Skipping unhashable information '
